# Satellite imagery collection and processing
---

Experimenting with filtering, downloading and displaying Sentinel 2 images from the [AWS STAC of Cloud-Optimized GeoTIFFs](https://registry.opendata.aws/sentinel-2-l2a-cogs/)

## Setup

### Imports

In [ ]:
from coal_emissions_monitoring.data_cleaning import (
    load_clean_campd_facilities_gdf,
    load_clean_campd_emissions_df,
    load_clean_image_metadata_df,
    get_final_dataset
)
from coal_emissions_monitoring.satellite_imagery import (
    create_aoi_for_plants,
    get_image_metadata_for_plants,
    get_image_from_cog
)
from coal_emissions_monitoring.data_viz import view_satellite_image

### Parameters

In [ ]:
cloud_filter_percent = 25

## Load CAMPD data

In [ ]:
campd_facilities_gdf = load_clean_campd_facilities_gdf("/Users/adminuser/GitHub/ccai-ss23-ai-monitoring-tutorial/data/facility_attributes.csv")
campd_facilities_gdf

In [ ]:
campd_facilities_gdf = create_aoi_for_plants(campd_facilities_gdf)
campd_facilities_gdf

In [ ]:
campd_facilities_gdf.geometry.explore()

In [ ]:
campd_emissions_df = load_clean_campd_emissions_df("/Users/adminuser/GitHub/ccai-ss23-ai-monitoring-tutorial/data/daily_emissions_facility_aggregation.csv")
campd_emissions_df

## Filter emissions data to days when a cloudless image is available

### Get image metadata for every power plant

In [ ]:
image_metadata_df = get_image_metadata_for_plants(campd_facilities_gdf, max_cloud_cover=cloud_filter_percent)
image_metadata_df.to_csv("/Users/adminuser/GitHub/ccai-ss23-ai-monitoring-tutorial/data/image_metadata.csv", index=False)
image_metadata_df

### Join with emissions data

In [ ]:
df = get_final_dataset(
    image_metadata_path="/home/adminuser/ccai-ss23-ai-monitoring-tutorial/data/image_metadata.csv",
    campd_facilities_path="https://drive.google.com/file/d/1b-5BriZUiiv2r0wFLubccLQpd2xb5ysl/view?usp=share_link",
    campd_emissions_path="https://drive.google.com/file/d/1oxZXR7GDcSXwwVoIjp66iS179cFVA5dP/view?usp=share_link",
    cog_type="all",
)
df

## Download and display images

In [ ]:
image = get_image_from_cog(cog_url=image_metadata_df.cog_url.iloc[0], geometry=campd_facilities_gdf.geometry.iloc[0])

In [ ]:
image.shape, image.min(), image.mean(), image.max()

In [ ]:
view_satellite_image(image)